In [278]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from multiprocessing import Pool
from tqdm import tqdm
import pickle
from datetime import datetime
from datetime import timedelta

Loading in all data for inspection:
0. data volume
1. field consistancy
2. missing values anywhere at all
3. data types
4. basic visualisation for relevant cols
5. etc

In [191]:
data_path = '/Users/ruoyangzhang/Documents/PythonWorkingDirectory/Eleven_super_case_customer_insights_Data/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

In [27]:
for file in onlyfiles:
    if file[:3] == 'LOG':
        print('the inspected log file is: ' + file)
    else:
        print('the inspected non log file is :' + file)
    print('it has: ', len(pd.read_csv('data/' + file, sep = ';', error_bad_lines = False)), 'lines')

the inspected non log file is :TABLE_CONVERSION.csv
it has:  580404 lines
the inspected non log file is :.DS_Store
it has:  0 lines
the inspected non log file is :CLIENTS.csv
it has:  3125800 lines
the inspected log file is: LOG_WEB_201710.csv
it has:  3301363 lines
the inspected log file is: LOG_WEB_201711.csv
it has:  5055599 lines
the inspected non log file is :CMD.csv
it has:  7323227 lines
the inspected log file is: LOG_WEB_201712.csv
it has:  4535451 lines
the inspected log file is: LOG_WEB_201708.csv
it has:  2189118 lines
the inspected log file is: LOG_WEB_201709.csv
it has:  2874217 lines
the inspected log file is: LOG_WEB_201801.csv


/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


it has:  8450825 lines
the inspected log file is: LOG_WEB_201802.csv
it has:  6063362 lines
the inspected log file is: LOG_WEB_201803.csv
it has:  2739719 lines


these files are rather large containing many lines, we should use a subset to test the robustness of the classification

In [194]:
data_201708 = pd.read_csv(data_path+'LOG_WEB_201708.csv', sep=';')

In [35]:
colns = sorted([col for col in list(data_201708.columns) if col[:3] != 'Unn'])

In [37]:
for file in onlyfiles:
    if file[:3] == 'LOG':
        colns_check = sorted([col for col in list(pd.read_csv('data/' + file, sep = ';').columns) if col[:3] != 'Unn'])
        print('the columns match the example columns: ', colns_check == colns)
        if colns_check != colns:
            print('the file with unmatched cols is: ', file)

the columns match the example columns:  True
the columns match the example columns:  True
the columns match the example columns:  True
the columns match the example columns:  True
the columns match the example columns:  True


/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


the columns match the example columns:  True
the columns match the example columns:  True
the columns match the example columns:  True


So the 201709 file is not matched, let's take a look at what's going on

In [195]:
data_201709 = pd.read_csv(data_path+'LOG_WEB_201709.csv', sep=';')

In [97]:
data_201709.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'VISITOR_ID', 'CONNECTED_SESSION',
       'ID_SESSION', 'SESSION_START_DATE', 'EVENT_DATE', 'PAGES',
       'GLOBAL_SOURCES', 'DEVICE_TYPE', 'DEVICE_MODEL', 'LOADINGS'],
      dtype='object')

In [54]:
set(data_201709.GLOBAL_SOURCES)

{'Accès Direct',
 'Affiliation et partenaires',
 'Email marketing',
 'Liens Sponsorisés',
 'Moteurs',
 'Notoriété',
 'Publicité',
 'Sites affluents',
 'Sources naturelles personnalisées',
 'Webmails'}

Let's take a look at the non log files

In [196]:
conver = pd.read_csv(data_path+'TABLE_CONVERSION_new.csv', sep = ';')

In [132]:
conver.head()

,Unnamed: 0,VISITOR_ID,CLIENT_NUMBER
0,0,153926,2117916268519517296
1,1,153987,-3983308646720649306
2,2,153992,-2525192206264180159
3,3,153996,1543724696132514059
4,4,154002,-3753347333864614269


In [197]:
clients = pd.read_csv(data_path+'CLIENTS_new.csv', sep = ';')

In [134]:
clients.head()

,Unnamed: 0,CLIENT_NUMBER,CREATION_DATE,GENDER,BIRTH_YEAR,ZIPCODE
0,0,-1182315409949716431,2002-12-05,Mme,1955.0,68440
1,1,7142641182482142162,2002-12-05,Mme,NaN,24100
2,2,-4786858050561650068,2002-12-05,Mme,1945.0,39240
3,3,-8463163259230597037,2002-12-05,Mme,NaN,42290
4,4,-8047395753631590665,2003-10-06,Mme,1934.0,13006


In [198]:
CMD = pd.read_csv(data_path+'CMD_new.csv', sep = ';')

In [136]:
CMD.head()

,Unnamed: 0,CLIENT_NUMBER,ORDER_NUMBER,ORDER_DATE,ORDER_CHANNEL,PRE_TAX_AMOUNT,CVIC
0,0,7178742953965764755,2302390,2010-01-05,Telephone,84.28,True
1,1,-2373390172893352743,2302392,2010-01-17,Telephone,97.85,True
2,2,-240944575938990062,2302393,2010-01-05,Telephone,58.45,True
3,3,1720395480962300217,2302394,2010-01-05,Telephone,31.19,True
4,4,7161131950067982732,2302396,2010-01-14,Telephone,37.45,True


In [51]:
set(CMD.ORDER_CHANNEL)

{'Courier', 'Internet', 'Magasin', 'Non Renseigne', 'Telephone'}

In [53]:
len(CMD.CLIENT_NUMBER), len(set(CMD.CLIENT_NUMBER))

(7323227, 1604670)

as expected, over the years, one clients made multiple purchases

As informed the supervisor, the CVIC column is the column that identifies the users who use promo codes

We have a general look around the order channel and the CVIC column

In [199]:
CMD = CMD.drop(['Unnamed: 0'], axis = 1)

In [200]:
len(CMD[['Internet' == channel for channel in CMD.ORDER_CHANNEL]]), len(CMD)

(1648148, 7323227)

In [64]:
CMD[['Internet' == channel for channel in CMD.ORDER_CHANNEL]].CVIC

633861     False
634168     False
634413     False
635627     False
636492     False
637017     False
637785     False
637833     False
638038     False
638045     False
1341334    False
1341945    False
1344864    False
1346825    False
1349597    False
1350807    False
1354861    False
1355086    False
1355754    False
1357128    False
1362250    False
1362834    False
1364217    False
1364964    False
1366807    False
1369690    False
2410345     True
2410346     True
2410347    False
2410348     True
           ...  
6724543     True
6724544     True
6724545     True
6724546     True
6724547    False
6724548    False
6724549    False
6724550    False
6724551    False
6724552    False
6724553    False
6724554    False
6724555     True
6724556    False
6724557    False
6724558    False
6724559    False
6724560     True
6724561    False
6724562    False
6724563     True
6724564     True
6724565    False
6724566    False
6724567     True
6724568     True
6724569    False
6724570    Fal

In [202]:
def desired_group(row):
    if row['CVIC'] == True:
        if row['ORDER_CHANNEL'] == 'Internet':
            if row['ORDER_DATE'][:4] == '2017':
                if int(row['ORDER_DATE'][5:7]) >= 8:
                    return(True)
            if row['ORDER_DATE'][:4] == '2018':
                return(True)

In [203]:
internet_promo_users = [(row['CLIENT_NUMBER'], row['ORDER_NUMBER'], row['ORDER_DATE'][:7]) for i, row in tqdm(CMD.iterrows()) if desired_group(row)]


7323227it [06:15, 19523.70it/s]


In [204]:
len(internet_promo_users)

51407

In [205]:
len(set([pair[0] for pair in internet_promo_users])),len(set([pair[1] for pair in internet_promo_users]))

(41804, 51407)

In [206]:
net_catalog_order = [pair[1] for pair in internet_promo_users]

In [207]:
net_catalog_client = list(set([pair[0] for pair in internet_promo_users]))

In [208]:
len(net_catalog_client)

41804

In [209]:
{identified_order_channels[i]:res_value_count[i] for i in range(len(identified_order_channels))}

{'Courier': 3543250,
 'Internet': 1648148,
 'Magasin': 727577,
 'Non Renseigne': 3971,
 'Telephone': 1400281}

We create a dico in order to be able to easily seperate the order ids and link them to the specific datasets

In [210]:
all_periods = list(set([tri[2] for tri in internet_promo_users]))

In [212]:
orders_by_date = {}
for tri in tqdm(internet_promo_users):
    if tri[2] not in orders_by_date.keys():
        orders_by_date[tri[2]] = []
    else:
        orders_by_date[tri[2]].append(tri[1])

100%|██████████| 51407/51407 [00:00<00:00, 1245388.41it/s]


In [213]:
client_by_date = {}
for tri in tqdm(internet_promo_users):
    if tri[2] not in client_by_date.keys():
        client_by_date[tri[2]] = []
    else:
        client_by_date[tri[2]].append(tri[0])

100%|██████████| 51407/51407 [00:00<00:00, 1253577.51it/s]


In [214]:
{k:len(v) for k,v in orders_by_date.items()}

{'2017-08': 5966,
 '2017-09': 4698,
 '2017-10': 5118,
 '2017-11': 5889,
 '2017-12': 4919,
 '2018-01': 6206,
 '2018-02': 3457,
 '2018-03': 5472,
 '2018-04': 4006,
 '2018-05': 4878,
 '2018-06': 787}

In [215]:
client_by_date = {date: list(set(clients)) for date, clients in client_by_date.items()}
{k:len(v) for k,v in client_by_date.items()}

{'2017-08': 5739,
 '2017-09': 4557,
 '2017-10': 4981,
 '2017-11': 5715,
 '2017-12': 4802,
 '2018-01': 6005,
 '2018-02': 3378,
 '2018-03': 5285,
 '2018-04': 3885,
 '2018-05': 4711,
 '2018-06': 775}

In [216]:
for key in all_periods:
    print(len(client_by_date[key]) <= len(orders_by_date[key]))

True
True
True
True
True
True
True
True
True
True
True


number of unique clients compared with number of unique orders is consistant, let's save this to a pickle for easier access later on

In [218]:
with open(data_path+'client_by_date.pickle', 'wb') as handle:
    pickle.dump(client_by_date, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(data_path+'orders_by_date.pickle', 'wb') as handle:
    pickle.dump(orders_by_date, handle, protocol=pickle.HIGHEST_PROTOCOL)

Loading in the pickles

In [219]:
with open(data_path+'client_by_date.pickle', 'rb') as handle:
    client_by_date = pickle.load(handle)
    
with open(data_path+'orders_by_date.pickle', 'rb') as handle:
    orders_by_date = pickle.load(handle)

Let's take a look at at least one month the clients behaviour to see if there is anything worth noting

We link the client numbers to the log files by the visitor ID variable in the conversion table

In [221]:
conver = conver.drop(['Unnamed: 0'], axis = 1)
conver.head()

,VISITOR_ID,CLIENT_NUMBER
0,153926,2117916268519517296
1,153987,-3983308646720649306
2,153992,-2525192206264180159
3,153996,1543724696132514059
4,154002,-3753347333864614269


In [224]:
conver_dict = {row['CLIENT_NUMBER']:row['VISITOR_ID'] for i, row in tqdm(conver.iterrows())}

580404it [00:27, 21395.51it/s]


Now let's extract the visitor ID of the net catalogue customers for 201709

In [239]:
net_cat_cn_1709 = client_by_date['2017-09']
net_cat_cn_1709_cross = set(net_cat_cn_1709)&set(conver.CLIENT_NUMBER)
net_cat_vid_1709_cross = [conver_dict[cn] for cn in net_cat_cn_1709_cross]

In [241]:
len(set(net_cat_cn_1709)&set(conver.CLIENT_NUMBER)), len(set(net_cat_cn_1709)), len(net_cat_cn_1709)

(3561, 4557, 4557)

Clearly there is some data discrepancies, some client numbers of internet catalogue customers are not found in the conversion table, for now we use the available ones' visitor ID for further investigation

In [243]:
data_201709 = data_201709.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)

In [244]:
data_201709.head(10)

,VISITOR_ID,CONNECTED_SESSION,ID_SESSION,SESSION_START_DATE,EVENT_DATE,PAGES,GLOBAL_SOURCES,DEVICE_TYPE,DEVICE_MODEL,LOADINGS
0,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:00:12,accueil,Email marketing,Ordinateur,NaN,1
1,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:07:23,accueil,Email marketing,Ordinateur,NaN,1
2,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:16:51,accueil,Email marketing,Ordinateur,NaN,1
3,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:19:24,accueil,Email marketing,Ordinateur,NaN,1
4,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:21:12,accueil,Email marketing,Ordinateur,NaN,1
5,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:24:19,accueil,Email marketing,Ordinateur,NaN,1
6,628575,OUI,17,2017-09-01 00:01:16,2017-09-01 00:28:05,accueil,Accès Direct,Ordinateur,NaN,1
7,628575,OUI,17,2017-09-01 00:01:16,2017-09-01 00:46:21,accueil,Accès Direct,Ordinateur,NaN,1
8,0,NON,19,2017-09-01 00:01:16,2017-09-01 00:01:16,accueil,Liens Sponsorisés,Ordinateur,NaN,1
9,0,NON,19,2017-09-01 00:01:16,2017-09-01 00:02:30,accueil,Liens Sponsorisés,Ordinateur,NaN,1


In [247]:
id_sesh_1709 = list(set(data_201709.ID_SESSION))

In [255]:
data_201709 = data_201709.reset_index(drop = True)

In [260]:
data_201709.index = [str(data_201709.ID_SESSION[i]) + str(data_201709.SESSION_START_DATE[i])[:10] for i in range(len(data_201709))]

In [264]:
data_201709.loc['32017-09-01', :]

,VISITOR_ID,CONNECTED_SESSION,ID_SESSION,SESSION_START_DATE,EVENT_DATE,PAGES,GLOBAL_SOURCES,DEVICE_TYPE,DEVICE_MODEL,LOADINGS
32017-09-01,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:00:12,accueil,Email marketing,Ordinateur,NaN,1
32017-09-01,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:07:23,accueil,Email marketing,Ordinateur,NaN,1
32017-09-01,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:16:51,accueil,Email marketing,Ordinateur,NaN,1
32017-09-01,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:19:24,accueil,Email marketing,Ordinateur,NaN,1
32017-09-01,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:21:12,accueil,Email marketing,Ordinateur,NaN,1
32017-09-01,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:24:19,accueil,Email marketing,Ordinateur,NaN,1
32017-09-01,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:09:23,category 17::fiche produit::713718043006883218,Email marketing,Ordinateur,NaN,1
32017-09-01,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:10:05,ajout au panier,Email marketing,Ordinateur,NaN,1
32017-09-01,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:12:02,ajout au panier,Email marketing,Ordinateur,NaN,1
32017-09-01,433347,OUI,3,2017-09-01 00:00:12,2017-09-01 00:13:23,ajout au panier,Email marketing,Ordinateur,NaN,1


In [365]:
unique_sessions = list(set(data_201709.index))
fields_unique_val = [ele for ele in list(data_201709.columns) if ele not in ['EVENT_DATE', 'PAGES']]

In [386]:
def return_dict_for_compression(unique_session_id):
    subdf = data_201709.loc[unique_session_id]
    
    if isinstance(subdf, pd.DataFrame):
        dict_to_return = {field:list(set(subdf[field]))[0] for field in fields_unique_val}
        event_dates = [datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in subdf.EVENT_DATE]
        time_before_final_page = (max(event_dates) - min(event_dates)).total_seconds()
        sesh_start_hour = list(set(subdf.SESSION_START_DATE))[0][11:13]
        unique_page_count = len(set(subdf.PAGES))
        landing_page = subdf.PAGES.iloc[0]
    else:
        dict_to_return = {field:subdf[field] for field in fields_unique_val}
        time_before_final_page = 0
        sesh_start_hour = subdf.SESSION_START_DATE[11:13]
        unique_page_count = 1
        landing_page = subdf.PAGES
    dict_to_return['time_before_final'] = time_before_final_page
    dict_to_return['start_hour'] = sesh_start_hour
    dict_to_return['nb_pages_unique'] = unique_page_count
    return(dict_to_return)

In [388]:
return_dict_for_compression(unique_sessions[0])

{'CONNECTED_SESSION': 'NON',
 'DEVICE_MODEL': nan,
 'DEVICE_TYPE': 'Ordinateur',
 'GLOBAL_SOURCES': 'Liens Sponsorisés',
 'ID_SESSION': 12276,
 'LOADINGS': 1,
 'SESSION_START_DATE': '2017-09-08 15:25:06',
 'VISITOR_ID': 0,
 'nb_pages_unique': 1,
 'start_hour': '15',
 'time_before_final': 0}

Since we're not sure of the function, we genrate the new dataframe

In [390]:
p = Pool(8, maxtasksperchild=1)
compresed_lines = p.map(return_dict_for_compression, tqdm(unique_sessions))
p.close()


 27%|██▋       | 117489/442324 [00:13<00:38, 8512.20it/s] 

KeyboardInterrupt: 

Process ForkPoolWorker-160:
Traceback (most recent call last):
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-386-9e9721395176>", line 2, in return_dict_for_compression
    subdf = data_201709.loc[unique_session_id]
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py", line 1373, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py", line 1627, in

  File "<ipython-input-386-9e9721395176>", line 2, in return_dict_for_compression
    subdf = data_201709.loc[unique_session_id]
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py", line 1373, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py", line 1627, in _getitem_axis
    return self._get_label(key, axis=axis)
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py", line 145, in _get_label
    return self.obj._xs(label, axis=axis)
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py", line 2344, in xs
    loc = self.index.get_loc(key)
  File "/Users/ruoyangzhang/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2525, in get_loc
    return self._engine.get_loc(key)
KeyboardInterrupt
Process ForkPoolWorker-158:

 29%|██▊       | 126384/442324 

In [395]:
p = Pool(8, maxtasksperchild=1)

p.close()